In [ ]:
import pyterrier as pt
import pyterrier_rag

sparse_index = pt.Artifact.from_hf('pyterrier/ragwiki-terrier')

# queries from R1 may have tokens that Terrier doesnt like. We can remove them and put them back later.
bm25 = pt.rewrite.tokenise() >> sparse_index.bm25(include_fields=['docno', 'text', 'title']) >> pt.rewrite.reset()

r1 = pyterrier_rag.R1Searcher(bm25)
print(r1.search("What are chemical reactions"))